In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.layers import Dense

2023-04-06 22:01:11.075081: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 22:01:12.528986: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
isa = pd.read_csv("/home/thesmilingturtle/coding/Python/Experiments/Shrinking/Tests/isa.csv", index_col='Name')
print(isa.head(3))

isa_tensor = tf.convert_to_tensor(isa)

isa_ds = tf.data.Dataset.from_tensor_slices(isa).batch(1)

      E1  E2  E3  E4  E5  E6  E7  E8  E9  E10  ...  O1  O2  O3  O4  O5  O6  \
Name                                           ...                           
Isa    2   2   5   1   5   2   5   3   4    1  ...   2   2   2   2   3   1   

      O7  O8  O9  O10  
Name                   
Isa    5   2   3    2  

[1 rows x 50 columns]


2023-04-06 22:01:15.992928: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

class Compressor(Model):
    def __init__(self):
        super(Compressor, self).__init__()
        self.l1 = Dense(50, input_shape=(50,))
        self.l2 = Dense(50, activation='relu')
        self.l3 = Dense(50, activation='tanh')
        self.l4 = Dense(20, activation='tanh')
        self.out = Dense(6, activation='tanh')

    def call(self, X):
        X = self.l1(X)
        X = self.l2(X)
        X = self.l3(X)
        X = self.l4(X)
        X = self.out(X)
        return X

class Decompressor(Model):
    def __init__(self):
        super(Decompressor, self).__init__()
        self.l1 = Dense(6, input_shape=(6,))
        self.l2 = Dense(40, activation='tanh')
        self.out = Dense(50)

    def call(self, X):
        X = self.l1(X)
        X = self.l2(X)
        X = self.out(X)
        return X
    
class Shrink(Model):
    def __init__(self):
        super(Shrink, self).__init__()
        self.compressor = Compressor()
        self.decompressor = Decompressor()

    def call(self, X):
        X = self.compressor(X)
        X = self.decompressor(X)
        return X


In [4]:
shrink = Shrink()

LOAD = '/home/thesmilingturtle/coding/Python/Experiments/Shrinking/Models/save2/save2'

shrink.load_weights(LOAD)

i = 0

loss_records = []
res = []

test_loss = tf.metrics.MeanAbsoluteError()

In [5]:
for person in isa_ds:
    test_loss.reset_states()

    compressed = shrink.compressor(person, training=False)
    decompressed = shrink.decompressor(compressed, training=False)

    test_loss(decompressed, person)
    loss_records.append(test_loss.result().numpy()-0.6242381930351257)

    res.append(compressed.numpy())

In [6]:
out_array = pd.DataFrame(np.vstack(res), index=['Isa'])
out_array['Uniqueness'] = loss_records
out_array

,0,1,2,3,4,5,Uniqueness
Isa,-0.018715,0.313005,-0.284812,-0.384875,0.294344,0.451311,0.000215


In [8]:
records = []

for person in isa_ds:
    compressed = shrink.compressor(person, training=False)
    decompressed = shrink.decompressor(compressed, training=False)

    records.append(tf.round(decompressed).numpy())

guessed_df = pd.DataFrame(np.vstack(records), index=['Isa'], columns=isa.columns, dtype=np.uint)

In [9]:
isa

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,...,O1,O2,O3,O4,O5,O6,O7,O8,O9,O10
Name,,,,,,,,,,,,,,,,,,,,,
Isa,2,2,5,1,5,2,5,3,4,1,...,2,2,2,2,3,1,5,2,3,2


In [10]:
guessed_df

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,...,O1,O2,O3,O4,O5,O6,O7,O8,O9,O10
Isa,4,1,5,1,5,1,5,2,4,1,...,3,3,3,3,3,2,4,2,3,3


In [31]:
dominik = pd.DataFrame([train.loc['Dominik'], guessed_df.loc['Dominik']], index=['actual', 'guessed'], dtype=np.uint)

In [32]:
dominik

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,...,O1,O2,O3,O4,O5,O6,O7,O8,O9,O10
actual,1,4,2,4,1,2,1,5,2,4,...,4,1,1,1,5,4,4,3,5,4
guessed,1,4,1,5,2,3,1,5,1,5,...,5,1,5,1,4,1,5,4,5,5


In [33]:
dominik.T.drop_duplicates(keep=False).T

,N5,N9,A3,O3,O6,O8
actual,4,2,1,1,4,3
guessed,3,4,3,5,1,4


In [34]:
vojta = pd.DataFrame([train.loc['Vojta'], guessed_df.loc['Vojta']], index=['actual', 'guessed'], dtype=np.uint)

In [35]:
vojta

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,...,O1,O2,O3,O4,O5,O6,O7,O8,O9,O10
actual,2,1,4,3,4,2,3,4,4,1,...,4,2,5,1,3,5,4,3,4,4
guessed,3,2,4,2,4,2,4,3,4,3,...,3,2,4,2,3,2,4,3,4,4


In [36]:
vojta.T.drop_duplicates(keep=False).T

,E10,O6
actual,1,5
guessed,3,2


In [53]:
null_person = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor( [[0, 0, 0, 0, 0, 0]]) ).batch(1)

In [59]:
for person in null_person:
    null_prediction = shrink.decompressor(person)

null_guy = pd.DataFrame(tf.round(null_prediction).numpy(), index=['null'], columns=train.columns)

In [60]:
null_guy

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,...,O1,O2,O3,O4,O5,O6,O7,O8,O9,O10
null,3.0,3.0,4.0,3.0,4.0,3.0,3.0,4.0,3.0,4.0,...,4.0,2.0,5.0,2.0,4.0,1.0,4.0,4.0,5.0,5.0


In [64]:
neg_person = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor( [[-1]*6]) ).batch(1)

In [65]:
for person in neg_person:
    neg_prediction = shrink.decompressor(person)

neg_guy = pd.DataFrame(tf.round(neg_prediction).numpy(), index=['neg'], columns=train.columns)

In [66]:
neg_guy

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,...,O1,O2,O3,O4,O5,O6,O7,O8,O9,O10
neg,4.0,2.0,5.0,4.0,4.0,3.0,4.0,4.0,3.0,5.0,...,2.0,6.0,4.0,5.0,3.0,3.0,3.0,1.0,4.0,3.0


In [70]:
pos_person = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor( [[1]*6]) ).batch(1)

In [71]:
for person in pos_person:
    pos_prediction = shrink.decompressor(person)

pos_guy = pd.DataFrame(tf.round(pos_prediction).numpy(), index=['pos'], columns=train.columns)

In [72]:
pos_guy

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,...,O1,O2,O3,O4,O5,O6,O7,O8,O9,O10
pos,-0.0,-1.0,-0.0,-0.0,1.0,-1.0,-1.0,-1.0,2.0,0.0,...,3.0,-1.0,3.0,-1.0,3.0,-2.0,3.0,2.0,2.0,3.0


In [76]:
test_loss.reset_states()

test_loss(vojta.loc['guessed'], vojta.loc['actual'])

test_loss.result() - 0.6242381930351257

<tf.Tensor: shape=(), dtype=float32, numpy=-0.02423817>